## Model Building

In [ ]:
import sagemaker
import numpy as np
import random

In [ ]:
# set random seeds for reproducibility
np.random.seed(42)
random.seed(42)

In [ ]:
sagemaker_session = sagemaker.Session()

In [ ]:
s3_bucket = ""
s3_prefix = "deepar_model"

#role = sagemaker.get_execution_role()
#role = ""

In [ ]:
region = sagemaker_session.boto_region_name

s3_data_path = "s3://{}/{}/data_start_poc".format(s3_bucket, s3_prefix)
s3_output_path = "s3://{}/{}/output_start_poc".format(s3_bucket, s3_prefix)

In [ ]:
image_name = sagemaker.image_uris.retrieve("forecasting-deepar", region)

In [ ]:
estimator = sagemaker.estimator.Estimator(
    image_uri = image_name,
    sagemaker_session = sagemaker_session,
    role = role,
    instance_count = 1,
    instance_type = "ml.c5.2xlarge",
    base_job_name = "deepar-poc-start",
    output_path = s3_output_path,
)

In [ ]:
freq = "15min"
context_length = 4 * 24 * 3
prediction_length = 4 * 24 * 3

In [ ]:
hyperparameters = {
    "time_freq": freq,
    "epochs": "400",
    "early_stopping_patience": "40",
    "mini_batch_size": "64",
    "learning_rate": "5E-4",
    "context_length": str(context_length),
    "prediction_length": str(prediction_length),
}

In [ ]:
estimator.set_hyperparameters(**hyperparameters)

In [ ]:
%%time
data_channels = {"train": "{}/train_start/".format(s3_data_path), "test": "{}/test_start/".format(s3_data_path)}

estimator.fit(inputs = data_channels, wait = True)